In [1]:
from tools import *
import pickle

## Add transported metabolites

In [2]:
def split_model_based_on_transport_rxns(model,tag):
    # model,   a metacyc based model
    # tag, for output file
    # 
    # Find all transport reactions in metacyc model. Save all these reactions into a new model.  
    # Remove those reactions from metacyc model
    #
    # Since there is no compartment information in the reaction, the metabolites transported are missing 
    # in the reaction because they exit in both side of the reaction. We will manually check the metacyc website
    # for those transporters and then add them back
    # 
    # check transport reaction in halo_meta
    # load transport reaction ids in metacyc database
    # The transport reactions were identified based on the transport reaction ids from RAVEN.
    is_transport = dict()
    for item in open('../../../ComplementaryData/metaCyc_transport_rxns.txt').read().split(): 
        is_transport[item] = True
    fhand = open('../Results/transport_rxns_{0}.tsv'.format(tag),'w')
    trmodel = cobra.Model('transmodel')
    non_trans_rxns = list()
    trans_rxns = list()
    k = 0
    for rxn in model.reactions:
        if is_transport.get(rxn.id,False): 
            k += 1
            print(k,rxn.id,rxn.name,':',rxn.reaction)
            fhand.write('{0}\t{1}\t{2}\t{3}\n'.format(k,rxn.id,rxn.name,rxn.reaction))
            trans_rxns.append(rxn.copy())
        else: non_trans_rxns.append(rxn.copy())
    trmodel.add_reactions(trans_rxns)
    for met in trmodel.metabolites: 
        try:met.charge = int(met.charge)
        except: met.charge=None
            
    fhand.close()

    ntrmodel = cobra.Model(model.id)
    ntrmodel.add_reactions(non_trans_rxns)
    
    return ntrmodel, trmodel

In [3]:
def the_TransRxns_For_Manual_Check(TransRxn_file):
    
    print('Ones have been manually checked')
    df_trans_old = pd.read_csv('../../../ComplementaryData/transport_rxns_mannual.tsv',
                               header=None,sep='\t',index_col=0)
    df_trans_old=df_trans_old.set_index(1)
    print(df_trans_old.shape)
    print(df_trans_old.head(5))
    print()
    
    print('Ones in the current model')
    df_trans_new = pd.read_csv(TransRxn_file,header=None,sep='\t',index_col=0)
    df_trans_new = df_trans_new.set_index(1)
    print(df_trans_new.head(5))
    print()
    
    print('Ones to be manually checked')
    print('''
    # if the below part is not empty, then add those to ../../../ComplementaryData/transport_rxns_mannual.tsv 
    # and manually collect transported metabolites from metacyc website
    ''')
    
    df_to_be_update = df_trans_new.loc[list(set(df_trans_new.index)-set(df_trans_old.index)),:]
    print(df_to_be_update)
    df_to_be_update.to_csv(TransRxn_file+'_to_be_manually_added.tsv',sep='\t',header=None)

In [4]:
def consider_all_rxns_happend_in_cytosal(model):
    for met in model.metabolites:
        met.id = met.id+'_c'
        met.compartment = 'c'
    model.repair()

In [5]:
def format_transported_met(rxn_id,transmodel,metacyc_model,direction,met_ids,weights):
    # rxn_id: rxn_id in transmodel
    # transmodel: the model containing all transport reactions
    # consider all transport reactions happen between cysotol and periplasm
    #
    # metacyc model: model reconstructed from MetaCyc
    # direction: 'in' or 'out'. 'in': p to c, out: c to p
    # met_ids: transported metabolite ids
    # weights: coeffs of tranported metabolite
    # 
    # Assume all metbolites in transport reactions before adding transported metabolit as in cyctosal.
    # 
    rxn = transmodel.reactions.get_by_id(rxn_id)
    #new_rxn = rxn.copy()
    
    # 1. add _c to exsiting metaboites in transport reaction
    coeffs = dict()
    for met in rxn.metabolites:
        new_id = met.id+'_c'

        try: new_met = metacyc_model.metabolites.get_by_id(new_id)
        except: 
            new_met = met.copy()
            new_met.id = new_met.id +'_c'
            new_met.compartment = 'c'
        coeffs[new_met] = rxn.get_coefficient(met)

    
    # 2. add transported metaboites
    def get_met_ec(met_id):
        try: met_c = metacyc_model.metabolites.get_by_id(met_id+'_c')
        except: 
            met_c = cobra.Metabolite(met_id+'_c')
            #met_c.name = met_id
            met_c.annotation = {'biocyc':'META:'+met_id,'id_source':'biocyc'}
            met_c.compartment = 'c'
            
            
        try: met_e = metacyc_model.metabolites.get_by_id(met_id+'_p')
        except: 
            met_e = cobra.Metabolite(met_id+'_p')
            met_e.annotation = met_c.annotation
            #met_e.name = met_id
            met_e.compartment = 'p'

        return met_e,met_c
    
    for i in range(len(met_ids)):
        met_id = met_ids[i]
        number_of_molecular = weights[i]
        met_e,met_c = get_met_ec(met_id)
        if direction == 'in': 
            coeffs[met_e] = coeffs.get(met_e,0)-number_of_molecular
            coeffs[met_c] = coeffs.get(met_c,0)+number_of_molecular
        elif direction == 'out':
            coeffs[met_c] = coeffs.get(met_c,0)-number_of_molecular
            coeffs[met_e] = coeffs.get(met_e,0)+number_of_molecular
        else: print('Error: check direction. ')

    
    new_rxn = rxn.copy()
    new_rxn.subtract_metabolites(new_rxn.metabolites)
    new_rxn.add_metabolites(coeffs)
    print(new_rxn.id,new_rxn.reaction)
    return new_rxn

In [6]:
def add_curated_transport_rxns(ntmodel,tmodel):
    trans_rxns_formated = list()
    trans_ids_to_be_add = [rxn.id for rxn in tmodel.reactions]
    for line in open('../../../ComplementaryData/transport_rxns_mannual.tsv'):
        cont = line.strip().split('\t')
        rxn_id = cont[1]
        if rxn_id not in trans_ids_to_be_add: continue
        direction = cont[5]
        met_ids = cont[4].split(',')
        weights = [int(item) for item in cont[6].split(',')]

        trans_rxns_formated.append(format_transported_met(rxn_id,tmodel,ntmodel,direction,
                                                      met_ids,weights)) 
    ntmodel.add_reactions(trans_rxns_formated)


In [7]:
def do_some_manual_curations(model):
    # curate the reversibility of several transport reactions
    print(model.reactions.get_by_id('TRANS-RXN0-244').reaction)
    model.reactions.get_by_id('TRANS-RXN0-244').lower_bound = 0
    print(model.reactions.get_by_id('TRANS-RXN0-244').reaction)
    
    print(model.reactions.get_by_id('TRANS-RXN0-200').reaction)
    model.reactions.get_by_id('TRANS-RXN0-200').upper_bound = 0
    print(model.reactions.get_by_id('TRANS-RXN0-200').reaction)
    
    
    # formation of NADPH is thermodynamically favored. Also in iML1515 THD2pp reaction
    # curate TRANS-RXN0-277
    # It should be NADP + NADH + H_p --> NADPH_c + NAD_c + H_c
    print(model.reactions.get_by_id('TRANS-RXN0-277').reaction)

    tmp_rxn = model.reactions.get_by_id('TRANS-RXN0-277')
    new_mets = {
        model.metabolites.get_by_id('NADPH_c'): 1,
        model.metabolites.get_by_id('NAD_c'): 1,
        model.metabolites.get_by_id('PROTON_c'): 1,
        model.metabolites.get_by_id('NADH_c'): -1,
        model.metabolites.get_by_id('NADP_c'): -1,
        model.metabolites.get_by_id('PROTON_p'): -1,
    }
    tmp_rxn.subtract_metabolites(tmp_rxn.metabolites)
    tmp_rxn.add_metabolites(new_mets)
    tmp_rxn.lower_bound = 0
    print(tmp_rxn.reaction)
    
    model.repair()


In [8]:
halo_meta = pickle.load(open('../Results/halo_metacycmodel_100_curated.pkl','rb'))
halo_mekg = pickle.load(open('../Results/halo_metacyc_kegg_curated.pkl','rb'))

Using license file /Users/gangl/gurobi.lic
Academic license - for non-commercial use only
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmptt3rhltl.lp
Reading time = 0.03 seconds
: 1553 rows, 2642 columns, 12000 nonzeros
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmppgu5bi5o.lp
Reading time = 0.02 seconds
: 2528 rows, 4478 columns, 20234 nonzeros


In [9]:
halo_meta_nt, halo_meta_t = split_model_based_on_transport_rxns(halo_meta,'halo_meta')

1 RXN-9615 ammonium transporter :  --> 
2 TRANS-RXN-214 Na(+)-translocating NADH-quinone reductase : NADH + PROTON + Ubiquinones --> NAD + Ubiquinols
3 RXN0-21 glutathione exporter : ATP + CPD-15815 --> ADP + PROTON + Pi
4 TRANS-RXN0-277 pyridine nucleotide transhydrogenase;NAD(P) transhydrogenase : NAD + NADPH <=> NADH + NADP
5 ABC-4-RXN L-arginine transporter : ATP + CPD-15815 --> ADP + PROTON + Pi
6 ABC-19-RXN molybdate ABC transporter;transport of molybdate : ATP + CPD-15815 --> ADP + PROTON + Pi
7 TRANS-RXN-347 norfloxacin:Na+ antiport :  <=> 
8 TRANS-RXN-237 transport of lipopolysaccharide : ATP + CPD-15815 --> ADP + PROTON + Pi
9 TRANS-RXN-167A D-glucosamine PTS permease : GLUCOSAMINE + Hpr-pi-phospho-L-histidines --> D-GLUCOSAMINE-6-P + Hpr-Histidine
10 TRANS-RXN0-513 lanthionine ABC transporter : ATP + CPD-15815 --> ADP + PROTON + Pi
11 TRANS-RXN-42 K+:H+ antiporter;K+ : H+ antiporter KefC;Na+/K+:H+ antiporter ChaA;K+ : H+ antiporter KefB :  <=> 
12 ABC-35-RXN L-leucine transp

In [10]:
halo_mekg_nt, halo_mekg_t = split_model_based_on_transport_rxns(halo_mekg,'halo_mekg')

1 RXN-9615 ammonium transporter :  --> 
2 TRANS-RXN-214 Na(+)-translocating NADH-quinone reductase : NADH + PROTON + Ubiquinones --> NAD + Ubiquinols
3 RXN0-21 glutathione exporter : ATP + CPD-15815 --> ADP + PROTON + Pi
4 TRANS-RXN0-277 pyridine nucleotide transhydrogenase;NAD(P) transhydrogenase : NAD + NADPH <=> NADH + NADP
5 ABC-4-RXN L-arginine transporter : ATP + CPD-15815 --> ADP + PROTON + Pi
6 ABC-19-RXN molybdate ABC transporter;transport of molybdate : ATP + CPD-15815 --> ADP + PROTON + Pi
7 TRANS-RXN-347 norfloxacin:Na+ antiport :  <=> 
8 TRANS-RXN-237 transport of lipopolysaccharide : ATP + CPD-15815 --> ADP + PROTON + Pi
9 TRANS-RXN-167A D-glucosamine PTS permease : GLUCOSAMINE + Hpr-pi-phospho-L-histidines --> D-GLUCOSAMINE-6-P + Hpr-Histidine
10 TRANS-RXN0-513 lanthionine ABC transporter : ATP + CPD-15815 --> ADP + PROTON + Pi
11 TRANS-RXN-42 K+:H+ antiporter;K+ : H+ antiporter KefC;Na+/K+:H+ antiporter ChaA;K+ : H+ antiporter KefB :  <=> 
12 ABC-35-RXN L-leucine transp

In [11]:
the_TransRxns_For_Manual_Check('../Results/transport_rxns_halo_meta.tsv')

Ones have been manually checked
(140, 5)
                                                                2  \
1                                                                   
RXN0-21                                      glutathione exporter   
RXN-9615                                     ammonium transporter   
TRANS-RXN0-479  selenite ABC transporter;selenite  ABC transpo...   
ABC-70-RXN       sulfate  ABC transporter;sulfate ABC transporter   
ABC-27-RXN                                 transport of phosphate   

                                                3            4    5  6  
1                                                                       
RXN0-21         ATP + WATER --> ADP + PROTON + Pi  GLUTATHIONE  out  1  
RXN-9615                                     -->      AMMONIUM   in  1  
TRANS-RXN0-479  ATP + WATER --> ADP + PROTON + Pi     SELENITE   in  1  
ABC-70-RXN      ATP + WATER --> ADP + PROTON + Pi      SULFATE   in  1  
ABC-27-RXN      ATP + WATER --> ADP +

In [12]:
the_TransRxns_For_Manual_Check('../Results/transport_rxns_halo_mekg.tsv')

Ones have been manually checked
(140, 5)
                                                                2  \
1                                                                   
RXN0-21                                      glutathione exporter   
RXN-9615                                     ammonium transporter   
TRANS-RXN0-479  selenite ABC transporter;selenite  ABC transpo...   
ABC-70-RXN       sulfate  ABC transporter;sulfate ABC transporter   
ABC-27-RXN                                 transport of phosphate   

                                                3            4    5  6  
1                                                                       
RXN0-21         ATP + WATER --> ADP + PROTON + Pi  GLUTATHIONE  out  1  
RXN-9615                                     -->      AMMONIUM   in  1  
TRANS-RXN0-479  ATP + WATER --> ADP + PROTON + Pi     SELENITE   in  1  
ABC-70-RXN      ATP + WATER --> ADP + PROTON + Pi      SULFATE   in  1  
ABC-27-RXN      ATP + WATER --> ADP +

In [13]:
consider_all_rxns_happend_in_cytosal(halo_mekg_nt)
consider_all_rxns_happend_in_cytosal(halo_meta_nt)

In [14]:
add_curated_transport_rxns(halo_meta_nt, halo_meta_t)

RXN0-21 ATP_c + CPD-15815_c + GLUTATHIONE_c --> ADP_c + GLUTATHIONE_p + PROTON_c + Pi_c
RXN-9615 AMMONIUM_p --> AMMONIUM_c
TRANS-RXN0-479 ATP_c + CPD-15815_c + SELENITE_p --> ADP_c + PROTON_c + Pi_c + SELENITE_c
ABC-70-RXN ATP_c + CPD-15815_c + SULFATE_p --> ADP_c + PROTON_c + Pi_c + SULFATE_c
ABC-27-RXN ATP_c + CPD-15815_c + Pi_p --> ADP_c + PROTON_c + 2.0 Pi_c
TRANS-RXN0-286 C6_c --> C6_p
TRANS-RXN0-268 ATP_c + CPD-15815_c + L-ALA-GAMMA-D-GLU-DAP_p --> ADP_c + L-ALA-GAMMA-D-GLU-DAP_c + PROTON_c + Pi_c
TRANS-RXN-355 CPD-9288_c + PROTON_c --> CPD-9288_p + PROTON_p
RXN0-11 ATP_c + CPD-15815_c + GLUTATHIONE_p --> ADP_c + GLUTATHIONE_c + PROTON_c + Pi_c
TRANS-RXN-167 Hpr-pi-phospho-L-histidines_c + N-acetyl-D-glucosamine_p --> Hpr-Histidine_c + N-ACETYL-D-GLUCOSAMINE-6-P_c
ABC-24-RXN ATP_c + CPD-15815_c + SPERMIDINE_p --> ADP_c + PROTON_c + Pi_c + SPERMIDINE_c
TRANS-RXN-352 DEOXYCHOLATE_c + PROTON_c --> DEOXYCHOLATE_p + PROTON_p
TRANS-RXN-296 ATP_c + COBINAMIDE_p + CPD-15815_c --> ADP_c +

In [15]:
add_curated_transport_rxns(halo_mekg_nt, halo_mekg_t)

RXN0-21 ATP_c + CPD-15815_c + GLUTATHIONE_c --> ADP_c + GLUTATHIONE_p + PROTON_c + Pi_c
RXN-9615 AMMONIUM_p --> AMMONIUM_c
TRANS-RXN0-479 ATP_c + CPD-15815_c + SELENITE_p --> ADP_c + PROTON_c + Pi_c + SELENITE_c
ABC-70-RXN ATP_c + CPD-15815_c + SULFATE_p --> ADP_c + PROTON_c + Pi_c + SULFATE_c
ABC-27-RXN ATP_c + CPD-15815_c + Pi_p --> ADP_c + PROTON_c + 2.0 Pi_c
TRANS-RXN0-286 C6_c --> C6_p
TRANS-RXN0-268 ATP_c + CPD-15815_c + L-ALA-GAMMA-D-GLU-DAP_p --> ADP_c + L-ALA-GAMMA-D-GLU-DAP_c + PROTON_c + Pi_c
TRANS-RXN-355 CPD-9288_c + PROTON_c --> CPD-9288_p + PROTON_p
RXN0-11 ATP_c + CPD-15815_c + GLUTATHIONE_p --> ADP_c + GLUTATHIONE_c + PROTON_c + Pi_c
TRANS-RXN-167 Hpr-pi-phospho-L-histidines_c + N-acetyl-D-glucosamine_p --> Hpr-Histidine_c + N-ACETYL-D-GLUCOSAMINE-6-P_c
ABC-24-RXN ATP_c + CPD-15815_c + SPERMIDINE_p --> ADP_c + PROTON_c + Pi_c + SPERMIDINE_c
TRANS-RXN-352 DEOXYCHOLATE_c + PROTON_c --> DEOXYCHOLATE_p + PROTON_p
TRANS-RXN-296 ATP_c + COBINAMIDE_p + CPD-15815_c --> ADP_c +

In [16]:
do_some_manual_curations(halo_meta_nt)

CD+2_c + PROTON_p --> CD+2_p + PROTON_c
CD+2_c + PROTON_p --> CD+2_p + PROTON_c
PROTON_c + ZN+2_p --> PROTON_p + ZN+2_c
PROTON_c + ZN+2_p --> PROTON_p + ZN+2_c
NADPH_c + NAD_c + PROTON_p <=> NADH_c + NADP_c + PROTON_c
NADH_c + NADP_c + PROTON_p --> NADPH_c + NAD_c + PROTON_c


In [17]:
do_some_manual_curations(halo_mekg_nt)

CD+2_c + PROTON_p --> CD+2_p + PROTON_c
CD+2_c + PROTON_p --> CD+2_p + PROTON_c
PROTON_c + ZN+2_p --> PROTON_p + ZN+2_c
PROTON_c + ZN+2_p --> PROTON_p + ZN+2_c
NADPH_c + NAD_c + PROTON_p <=> NADH_c + NADP_c + PROTON_c
NADH_c + NADP_c + PROTON_p --> NADPH_c + NAD_c + PROTON_c


In [18]:
report_model_status(halo_meta)

Number of reactions: 1321
Number of metabolits: 1553
Number of compartments: 1 {'s': ''}
Number of genes: 1252
Number of missing genes: 0
Number of reactions with missing genes: 0



In [19]:
report_model_status(halo_meta_nt)

Number of reactions: 1321
Number of metabolits: 1676
Number of compartments: 2 {'c': '', 'p': ''}
Number of genes: 1252
Number of missing genes: 0
Number of reactions with missing genes: 0



In [20]:
report_model_status(halo_mekg)

Number of reactions: 2239
Number of metabolits: 2528
Number of compartments: 1 {'s': ''}
Number of genes: 1326
Number of missing genes: 0
Number of reactions with missing genes: 0



In [21]:
report_model_status(halo_mekg_nt)

Number of reactions: 2239
Number of metabolits: 2651
Number of compartments: 2 {'c': '', 'p': ''}
Number of genes: 1326
Number of missing genes: 0
Number of reactions with missing genes: 0



## Save model

In [22]:
pickle.dump(refresh_model(halo_meta_nt),open('../Results/halo_metacycmodel_100_curated_compart.pkl','wb'))

Number of reactions: 1321
Number of metabolits: 1676
Number of compartments: 2 {'c': '', 'p': ''}
Number of genes: 1252
Number of missing genes: 0
Number of reactions with missing genes: 0



In [23]:
pickle.dump(refresh_model(halo_mekg_nt),open('../Results/halo_metacyc_kegg_curated_compart.pkl','wb'))

Number of reactions: 2239
Number of metabolits: 2651
Number of compartments: 2 {'c': '', 'p': ''}
Number of genes: 1326
Number of missing genes: 0
Number of reactions with missing genes: 0

